In [1]:
from serpapi import GoogleSearch
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os

# === USER INPUT SECTION ===
industry_input = input("Enter industry (e.g., IT, finance, marketing): ").strip()
company_size_input = input("Enter company size (e.g., 20-50, 100-200): ").strip()
location_input = input("Enter location (e.g., India, USA): ").strip()

# === WEB SCRAPING USING SERPAPI ===
api_key = "2edc54b008c95ddedc0fcd9b1b19601cf3194f160ccb50eca0ee1f42dbd98d66"
query = f"{industry_input} companies in {location_input} with {company_size_input} employees site:linkedin.com"

params = {
    "engine": "google",
    "q": query,
    "api_key": api_key,
    "num": "20"
}

search = GoogleSearch(params)
results = search.get_dict()
structured_data = []

for result in results.get("organic_results", []):
    name = result.get("title", "")
    url = result.get("link", "")
    snippet = result.get("snippet", "")

    email = ""
    phone = ""
    has_contact = False
    is_relevant = False
    website_verified = False

    try:
        response = requests.get(url, timeout=5)
        website_verified = True
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text()

        emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", text)
        phones = re.findall(r"\b\d{10,13}\b", text)

        email = emails[0] if emails else ""
        phone = phones[0] if phones else ""
        has_contact = bool(email or phone)

        if industry_input.lower() in text.lower():
            is_relevant = True

    except Exception as e:
        print(f"❌ Failed to fetch {url}: {e}")

    company_entry = {
        "Company Name": name,
        "Website": url,
        "Description": snippet,
        "Industry": industry_input,
        "Company Size": company_size_input,
        "Location": location_input,
        "Email": email,
        "Phone": phone,
        "Other Criteria - Has Contact": "Yes" if has_contact else "No",
        "Other Criteria - Verified Website": "Yes" if website_verified else "No",
        "Other Criteria - Relevant Industry": "Yes" if is_relevant else "No"
    }

    structured_data.append(company_entry)
    time.sleep(1)

lead_df = pd.DataFrame(structured_data)
lead_df.to_excel("Final_IT_Companies_with_ICP.xlsx", index=False)
print("✅ Data saved to Final_IT_Companies_with_ICP.xlsx")

# === SEND TEST EMAIL TO PROVIDED EMAIL ===
print("\n📤 Sending test emails to khushalimakani02@gmail.com...")

receiver_email = "parmarparmar1298@gmail.com"
with open("C:/Users/parma/OneDrive/Desktop/sales_automation_project/email_template.html", "r", encoding="utf-8") as f:
    html_template = f.read()

smtp_server = "smtp-relay.brevo.com"
smtp_port = 587
smtp_username = "8aa752001@smtp-brevo.com"
smtp_password = "tv5SfWQrPDhU3V4O"
sender_email = "gaurang.kology@gmail.com"

server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()
server.login(smtp_username, smtp_password)

# Limit to first 5 companies
for index, row in lead_df.head(5).iterrows():
    company_name = row.get("Company Name", "Demo Company")
    recipient_name = company_name.split()[0] if isinstance(company_name, str) else "there"

    email_body = html_template.replace("[Recipient Name]", recipient_name)
    email_body = email_body.replace("[Company Name]", company_name)
    email_body = email_body.replace("[Email]", receiver_email)

    msg = MIMEMultipart("alternative")
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = f"[TEST] Hello from {company_name}!"
    msg["Bcc"] = sender_email
    msg.attach(MIMEText(email_body, "html"))

    try:
        server.sendmail(sender_email, [receiver_email, sender_email], msg.as_string())
        print(f"✅ Sent test email to {receiver_email} ({company_name})")
        time.sleep(3)
    except Exception as e:
        print(f"❌ Failed to send to {receiver_email}: {e}")

server.quit()


Enter industry (e.g., IT, finance, marketing):  IT
Enter company size (e.g., 20-50, 100-200):  20
Enter location (e.g., India, USA):  india


❌ Failed to fetch https://www.linkedin.com/pulse/top-10-biggest-service-companies-india-farhan-ali-9gw0f: HTTPSConnectionPool(host='www.linkedin.com', port=443): Read timed out. (read timeout=5)
❌ Failed to fetch https://www.linkedin.com/pulse/20-top-full-stack-development-companies-india-2025-nln4c/: HTTPSConnectionPool(host='www.linkedin.com', port=443): Read timed out. (read timeout=5)
❌ Failed to fetch https://www.linkedin.com/pulse/20-top-web-development-services-companies-india-next-big-technology-xu9bc: HTTPSConnectionPool(host='www.linkedin.com', port=443): Read timed out. (read timeout=5)
❌ Failed to fetch https://www.linkedin.com/pulse/20-top-php-development-companies-india-2025-next-big-technology-otz9c: HTTPSConnectionPool(host='www.linkedin.com', port=443): Read timed out. (read timeout=5)
❌ Failed to fetch https://www.linkedin.com/pulse/top-staff-augmentation-companies-india-2025-haan-emani-blkmf: HTTPSConnectionPool(host='www.linkedin.com', port=443): Read timed out. (re


KeyboardInterrupt

